In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from PIL import Image
import io
from transformers import AutoModelForCausalLM,BitsAndBytesConfig
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16,
   llm_int8_skip_modules= ["resampler"]
)

model = AutoModelForCausalLM.from_pretrained("AIDC-AI/Ovis1.6-Gemma2-9B",
                                             torch_dtype=torch.bfloat16,
                                             multimodal_max_length=8192,
                                             device_map="auto", 
                                             quantization_config=nf4_config,
                                             trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
pil_image = Image.open("temiz.png")

In [6]:
text_tokenizer = model.get_text_tokenizer()
visual_tokenizer = model.get_visual_tokenizer()

In [7]:
prompt = "bu nedir"
query = f'<image>\n{prompt}'
prompt, input_ids, pixel_values = model.preprocess_inputs(query, [pil_image])
attention_mask = torch.ne(input_ids, text_tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device, dtype=torch.bfloat16)  
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
pixel_values = pixel_values.unsqueeze(0).to(dtype=torch.bfloat16, device=model.device)